In [1]:
#loading packages
import pyodbc
import csv
import pandas as pd
import sqlalchemy as sa
import psycopg2
import string

In [2]:
#local file location
fileLoc = r'C:\Users\hcole\Desktop\cfbstats-com-2005-1-5-0\\'


#loading files into dataframes
dfConference = pd.read_csv(fileLoc + 'conference.csv')
dfStadium = pd.read_csv(fileLoc + 'stadium.csv')
dfTeam = pd.read_csv(fileLoc + 'team.csv')
dfGame = pd.read_csv(fileLoc + 'game.csv')
dfGameStatistics = pd.read_csv(fileLoc + 'game-statistics.csv')
dfPlayer = pd.read_csv(fileLoc + 'player.csv')
dfPlayerGameStatistics = pd.read_csv(fileLoc + 'player-game-statistics.csv')
dfTeamGameStatistics = pd.read_csv(fileLoc + 'team-game-statistics.csv')
dfDrive = pd.read_csv(fileLoc + 'drive.csv')
dfPlay = pd.read_csv(fileLoc + 'play.csv')
dfPunt = pd.read_csv(fileLoc + 'punt.csv')
dfPuntReturn = pd.read_csv(fileLoc + 'punt-return.csv')
dfReception = pd.read_csv(fileLoc + 'reception.csv')
dfRush = pd.read_csv(fileLoc + 'rush.csv')
dfKickoff = pd.read_csv(fileLoc + 'kickoff.csv')
dfKickoffReturn = pd.read_csv(fileLoc + 'kickoff-return.csv')
dfPass = pd.read_csv(fileLoc + 'pass.csv')

#creating database connection string
conn = psycopg2.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=test2 user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
cur = conn.cursor()

In [3]:
#dataframe file array
dfFileNameCleaned = ['conference_cleaned', 'stadium_cleaned', 'stadiumdetails_cleaned', 'team_cleaned', 'teamdetails_cleaned', 'game_cleaned',
                     'gamestatistics_cleaned', 'player_cleaned', 'playerdetails_cleaned', 'playergamestatistics_cleaned',
                     'teamgamestatistics_cleaned', 'drive_cleaned', 'play_cleaned', 'punt_cleaned', 'puntreturn_cleaned', 'reception_cleaned',
                     'rush_cleaned', 'kickoff_cleaned', 'kickoffreturn_cleaned', 'pass_cleaned']

dfTableNames = ['conference', 'stadium', 'stadiumdetails', 'team', 'teamdetails', 'game', 'gamestatistics', 'player', 'playerdetails',
                'playergamestatistics', 'teamgamestatistics', 'drive', 'play', 'punt', 'puntreturn', 'reception', 'rush', 'kickoff',
                'kickoffreturn', 'pass']

#cleaning dataframes
for df in (dfConference, dfStadium, dfTeam, dfGame, dfGameStatistics, dfPlayer, dfPlayerGameStatistics, dfTeamGameStatistics, dfDrive, dfPlay, dfPunt, dfPuntReturn, dfReception, dfRush, dfKickoff, dfKickoffReturn, dfPass):
    df.columns = df.columns.str.replace('[^\w\s]', '')
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str.replace('1st', 'First')
    df.columns = df.columns.str.lower()

dfStadiumDetails = dfStadium[['name', 'city', 'state', 'capacity', 'surface', 'stadiumcode']]
dfStadium = dfStadium[['stadiumcode', 'yearopened']]

dfTeamDetails = dfTeam[['teamcode', 'conferencecode']]
dfTeam = dfTeam[['teamcode', 'name']]

dfPlayerDetails = dfPlayer[['playercode', 'teamcode', 'uniformnumber', 'class', 'position', 'height', 'weight', 'lastschool']]
dfPlayer = dfPlayer[['playercode', 'lastname', 'firstname', 'hometown', 'homestate', 'homecountry']]
dfPlayer['firstname'] = dfPlayer['firstname'].str.replace('[^\w\s]','')
dfPlayer['lastname'] = dfPlayer['lastname'].str.replace('[^\w\s]','') 

dfPass['passerplayercode'] = dfPass['passerplayercode'].astype(int)
dfPass['receiverplayercode'] = dfPass['passerplayercode'].astype(int)


dfFinalArray = [dfConference, dfStadium, dfStadiumDetails, dfTeam, dfTeamDetails, dfGame, dfGameStatistics, dfPlayer, dfPlayerDetails,
                dfPlayerGameStatistics, dfTeamGameStatistics, dfDrive, dfPlay, dfPunt, dfPuntReturn, dfReception, dfRush, dfKickoff, dfKickoffReturn, dfPass]

In [4]:
for df, file, table in zip(dfFinalArray, dfFileNameCleaned, dfTableNames):
    print(df.shape)
    print(file)
    print(table)
    
    df.to_csv(r'C:\Users\hcole\Desktop\cleaned\\' + file + '.csv', index=False)
    fileString = r'C:\Users\hcole\Desktop\cleaned\\' + file + '.csv'
    with open(fileString, 'r') as f:
    # Notice that we don't need the `csv` module.
        next(f)  # Skip the header row.
        cur = conn.cursor()
        cur.copy_from(f, table, sep=',')
        conn.commit()

(27, 3)
conference_cleaned
conference
(144, 2)
stadium_cleaned
stadium
(144, 6)
stadiumdetails_cleaned
stadiumdetails
(239, 2)
team_cleaned
team
(239, 2)
teamdetails_cleaned
teamdetails
(718, 6)
game_cleaned
game
(718, 3)
gamestatistics_cleaned
gamestatistics
(22879, 6)
player_cleaned
player
(22879, 8)
playerdetails_cleaned
playerdetails
(49257, 58)
playergamestatistics_cleaned
playergamestatistics
(1436, 68)
teamgamestatistics_cleaned
teamgamestatistics
(18876, 15)
drive_cleaned
drive
(135288, 14)
play_cleaned
play
(7525, 11)
punt_cleaned
punt
(4511, 11)
puntreturn_cleaned
puntreturn
(26563, 11)
reception_cleaned
reception
(55221, 12)
rush_cleaned
rush
(7631, 12)
kickoff_cleaned
kickoff
(4902, 11)
kickoffreturn_cleaned
kickoffreturn
(45887, 12)
pass_cleaned
pass
